<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #FF1493; background-color: #ffffff;">Bristol-Myers Squibb – Molecular Translation</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">Exploratory Data Analysis (EDA)</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>

---

<br>


<h1 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;">TABLE OF CONTENTS</h1>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#background_information">1&nbsp;&nbsp;&nbsp;&nbsp;BACKGROUND INFORMATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#setup">2&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">3&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#baseline">4&nbsp;&nbsp;&nbsp;&nbsp;BASELINE</a></h3>

<h1 style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;" id="imports">0&nbsp;&nbsp;IMPORTS</h1>

In [ ]:
# FOR KERAS UTILS PLOT
!pip install -q pydot
!pip install -q pydotplus
!apt-get install -q graphviz
from Levenshtein import distance

print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")

# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import tqdm
import time
import gzip
import ast
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2


print("\n\n... IMPORTS COMPLETE ...\n")


print("\n\n... TPU SETUP STARTING ...\n")
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except ValueError: # no TPU found, detect GPUs
    strategy = tf.distribute.get_strategy() # for GPU or multi-GPU machines
    print("\n... USING GPU ...\n")
    # Stop Tensorflow From Eating All The Memory
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "... Physical GPUs,", len(logical_gpus), "Logical GPUs ...\n")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)
        
N_REPLICAS = strategy.num_replicas_in_sync
print(f"... Number Of Accelerators: {N_REPLICAS} ...\n")
print("\n\n... TPU SETUP COMPLETE COMPLETE ...\n")

<h1 style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="background_information">1&nbsp;&nbsp;BACKGROUND INFORMATION</h1>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.1  THE DATA</h3>

---

<b style="text-decoration: underline; font-family: Verdana;">BACKGROUND INFORMATION</b>

In this competition, you are provided with images of chemicals, with the objective of predicting the corresponding [**International Chemical Identifier**](https://en.wikipedia.org/wiki/International_Chemical_Identifier) (**InChI**) text string of the image. The images provided (both in the training data as well as the test data) may be rotated to different angles, be at various resolutions, and have different noise levels.
* Possibilities for augmentation via rotation and noise
* There are about 4 Million total images in this dataset. Unzipping the downloaded data will take a non-trivial amount of time.

<br>

***Author's Take:***<br><br>
&nbsp;&nbsp;&nbsp;&nbsp;**1. Identify Feature Vector From Image of Chemical Structure**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**2. Generate the Required String** 

<br>

Submissions are evaluated on the mean Levenshtein distance between the InChi strings you submit and the ground truth InChi values.

For each **image_id** in the test set, **you must predict the InChi string of the molecule in the corresponding image**. The file should contain a header and have the following format:

```txt
image_id,InChI
00000d2a601c,InChI=1S/H2O/h1H2
00001f7fc849,InChI=1S/H2O/h1H2
000037687605,InChI=1S/H2O/h1H2
etc.
```

<br><br>

<b style="text-decoration: underline; font-family: Verdana;">DATA FILES & DIRECTORIES</b>
> **`train/`** - the training images, arranged in a 3-level folder structure by **image_id**<br>
> **`test/`** - the test images, arranged in the same folder structure as **`train/`**<br>
> **`train_labels.csv`** - ground truth **InChi** labels for the training images<br>
> **`sample_submission.csv`** - a sample submission file in the correct format<br>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.2  INTERNATIONAL CHEMICAL IDENTIFIER (InChi)</h3>

<sub>[***Most information is from this wikipedia page***](https://en.wikipedia.org/wiki/International_Chemical_Identifier)</sub>

---

The IUPAC International Chemical Identifier *(InChI /ˈɪntʃiː/ IN-chee or /ˈɪŋkiː/ ING-kee)* is a textual identifier for chemical substances, designed to provide a standard way to encode molecular information and to facilitate the search for such information in databases and on the web. Initially developed by **IUPAC** (**I**nternational **U**nion of **P**ure and **A**pplied **C**hemistry) and **NIST** (**N**ational **I**nstitute of **S**tandards and **T**echnology) from 2000 to 2005, the format and algorithms are non-proprietary.

The continuing development of the standard has been supported since 2010 by the not-for-profit **InChI Trust**, of which **IUPAC** is a member. The current software version is 1.06 and was released in December 2020.<br>
&nbsp;&nbsp;&nbsp;&nbsp;– Prior to 1.04, the software was freely available under the open-source LGPL license.<br>
&nbsp;&nbsp;&nbsp;&nbsp;– It now uses a custom license called IUPAC-InChI Trust License.

<br><br>

<b style="text-decoration: underline; font-family: Verdana;">OVERVIEW</b>

The identifiers describe chemical substances in terms of **layers of information**<br>
&nbsp;&nbsp;&nbsp;&nbsp;– The Atoms and Their Bond Connectivity<br>
&nbsp;&nbsp;&nbsp;&nbsp;– [Tautomeric Information](https://en.wikipedia.org/wiki/Tautomer)<br>
&nbsp;&nbsp;&nbsp;&nbsp;– [Isotope Information](https://en.wikipedia.org/wiki/Isotope)<br>
&nbsp;&nbsp;&nbsp;&nbsp;– [Stereochemistry](https://en.wikipedia.org/wiki/Stereochemistry)<br>
&nbsp;&nbsp;&nbsp;&nbsp;– [Electronic Charge Information](https://en.wikipedia.org/wiki/Electric_charge)<br>

**NOTE:**
Not all layers have to be provided; for instance, the tautomer layer can be omitted if that type of information is not relevant to the particular application.

**InChIs** differ from the widely used CAS registry numbers in three respects:<br>
&nbsp;&nbsp;&nbsp;&nbsp;– **firstly**, they are freely usable and non-proprietary;<br>
&nbsp;&nbsp;&nbsp;&nbsp;– **secondly**, they can be computed from structural information and do not have to be assigned by some organization;<br>
&nbsp;&nbsp;&nbsp;&nbsp;– **thirdly**, most of the information in an **InChI** is human readable (with practice).

InChIs can thus be seen as akin to a general and extremely formalized version of IUPAC names. They can express more information than the simpler SMILES notation and differ in that every structure has a unique InChI string, which is important in database applications. Information about the 3-dimensional coordinates of atoms is not represented in InChI; for this purpose a format such as PDB can be used.

<br>

---

<br>

The InChI algorithm converts input structural information into a unique InChI identifier in a three-step process: 
1. **Normalization** *(to remove redundant information)*
2. **Canonicalization** *(to generate a unique number label for each atom)*
3. **Serialization** *(to give a string of characters)*


<br>

---

<br>

In order to avoid generating different InChIs for tautomeric structures, before generating the InChI, an input chemical structure is normalized to reduce it to its so-called core parent structure. This may involve changing bond orders, rearranging formal charges and possibly adding and removing protons. Different input structures may give the same result; 
- For example, acetic acid and acetate would both give the same core parent structure, that of acetic acid. 
- A core parent structure may be disconnected, consisting of more than one component, in which case the sublayers in the InChI usually consist of sublayers for each component, separated by semicolons (periods for the chemical formula sublayer.) One way this can happen is that all metal atoms are disconnected during normalization;
    - So, for example, the InChI for tetraethyllead will have five components, one for lead and four for the ethyl groups.

The first, main, layer of the InChI refers to this core parent structure, giving its chemical formula, non-hydrogen connectivity without bond order (/c sublayer) and hydrogen connectivity (/h sublayer.) The /q portion of the charge layer gives its charge, and the /p portion of the charge layer tells how many protons (hydrogen ions) must be added to or removed from it to regenerate the original structure. If present, the stereochemical layer, with sublayers /b, /t, /m and /s, gives stereochemical information, and the isotopic layer /i (which may contain sublayers /h, /b, /t, /m and /s) gives isotopic information. These are the only layers which can occur in a standard InChI.[5]

If the user wants to specify an exact tautomer, a fixed hydrogen layer /f can be appended, which may contain various additional sublayers; this cannot be done in standard InChI though, so different tautomers will have the same standard InChI (for example, alanine will give the same standard InChI whether input in a neutral or a zwitterionic form.) Finally, a nonstandard reconnected /r layer can be added, which effectively gives a new InChI generated without breaking bonds to metal atoms. This may contain various sublayers, including /f.[5]

<br>

---

<br>

**STRUCTURE**

- Every InChI starts with the string "InChI=" followed by the version number, currently 1. 
- If the InChI is standard, this is followed by the letter S for standard InChIs, which is a fully standardized InChI flavor maintaining the same level of attention to structure details and the same conventions for drawing perception. 
- The remaining information is structured as a sequence of layers and sub-layers, with each layer providing one specific type of information. 
    * The layers and sub-layers are separated by the delimiter "/" and start with a characteristic prefix letter (except for the chemical formula sub-layer of the main layer). 
    * The six layers with important sublayers are:

<br>

>**1. Main Layer**
* Chemical formula (no prefix). This is the only sublayer that must occur in every InChI.
* Atom connections (prefix: `"c"`). 
    * The atoms in the chemical formula (except for hydrogens) are numbered in sequence. This sublayer describes which atoms are connected by bonds to which other ones.
* Hydrogen atoms (prefix: `"h"`). 
    * Describes how many hydrogen atoms are connected to each of the other atoms.

<br>

> **2. Charge Layer**
* Charge Sublayer (prefix: `"q"`)
* Proton Sublayer (prefix: `"p"`)

<br>

> **3. Stereochemical Layer**
* Double Bonds and Cumulenes (prefix: `"b"`)
* Tetrahedral Stereochemistry of Atoms and Allenes (prefixes: `"t"`, `"m"`)
* Type of Stereochemistry Information (prefix: `"s"`)

<br>

> **4. Isotopic Layer**
* prefixes: `"i"`, `"h"`, as well as `"b"`, `"t"`, `"m"`, `"s"` for isotopic stereochemistry

<br>

> **5. Fixed-H Layer**
* prefix: `"f"`. **NEVER INCLUDED IN STANDARD INCHI**

<br>

> **6. Reconnected Layer**
* prefix: `"r"`. **NEVER INCLUDED IN STANDARD INCHI**

<br>

---


***UNRELATED ASIDE***

---

The InChIKey, sometimes referred to as a hashed InChI, is a fixed length (27 character) condensed digital representation of the InChI that is not human-understandable. The InChIKey specification was released in September 2007 in order to facilitate web searches for chemical compounds, since these were problematic with the full-length InChI.[6] Unlike the InChI, the InChIKey is not unique: though collisions can be calculated to be very rare, they happen.[7]

---

<h1 style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="setup">2&nbsp;&nbsp;SETUP</h1>

In [ ]:
ALL_ELEMENTS = ["H", " He", " Li", " Be", " B", " C", " N", " O", " F", " Ne", " Na", " Mg", " Al", " Si", " P", 
                "S", " Cl", " Ar", " K", " Ca", " Sc", " Ti", " V", " Cr", " Mn", " Fe", " Co", " Ni", " Cu", " Zn", 
                "Ga", " Ge", " As", " Se", " Br", " Kr", " Rb", " Sr", " Y", " Zr", " Nb", " Mo", " Tc", " Ru", 
                "Rh", " Pd", " Ag", " Cd", " In", " Sn", " Sb", " Te", " I", " Xe", " Cs", " Ba", " La", " Ce", 
                "Pr", " Nd", " Pm", " Sm", " Eu", " Gd", " Tb", " Dy", " Ho", " Er", " Tm", " Yb", " Lu", " Hf", 
                "Ta", " W", " Re", " Os", " Ir", " Pt", " Au", " Hg", " Tl", " Pb", " Bi", " Po", " At", " Rn", 
                "Fr", " Ra", " Ac", " Th", " Pa", " U", " Np", " Pu", " Am", " Cm", " Bk", " Cf", " Es", " Fm", 
                "Md", " No", " Lr", " Rf", " Db", " Sg", " Bh", " Hs", " Mt", " Ds", " Rg", " Cn", " Uut", 
                "Fl", " Uup", " Lv", " Uus", " Uuo"]

# In Order
TRAIN_ELEMENTS = ['C', 'H', 'B', 'Br', 'Cl', 'F', 'I', 'N', 'O', 'P', 'S', 'Si']

# Prefixes and How Common
#      -- ORDERING --> {c}{h/None}{b/None}{t/None}{m/None}{s/None}{i/None}{h/None}{t/None}{m/None}
PREFIX_ORDERING = "chbtmsihtm"

# Define the root and data directories
ROOT_DIR = "/kaggle/input"

# try:
#     DATA_DIR = KaggleDatasets().get_gcs_path("bms-molecular-translation")
#     print("USING TPU (GCS) DATA DIRECTORY")
# except:
#     print("NOT USING TPU")
DATA_DIR = os.path.join(ROOT_DIR, "bms-molecular-translation")

TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

TRAIN_CSV_PATH = os.path.join(DATA_DIR, "train_labels.csv")
SS_CSV_PATH = os.path.join(DATA_DIR, "sample_submission.csv")

train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df["img_path"] = train_df.image_id.apply(lambda x: os.path.join(TRAIN_DIR, x[0], x[1], x[2], x+".png"))
print("\n... TRAIN DATAFRAME W/ PATHS ...\n")
display(train_df)

ss_df = pd.read_csv(SS_CSV_PATH)
print("\n... SUBMISSION DATAFRAME ...\n")
display(ss_df)

<h1 style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="helper_functions">3&nbsp;&nbsp;HELPER FUNCTIONS</h1>

In [ ]:
def flatten_l_o_l(nested_list):
    return [item for sublist in nested_list for item in sublist]

def get_sublayer(split_list, prefix="c"):
    if not split_list:
        return -1
    else:
        return split_list.pop(0)[1:]
    
    
def get_sublayer_hard(split_list, prefix="t"):
    if not split_list:
        pass
    else:
        inchi_idx_list = [i for i,inchi_str in enumerate(split_list) if prefix in inchi_str]
        if inchi_idx_list:
            return split_list.pop(inchi_idx_list[0])[1:]
        else:
            pass
        
        
# TRAIN --> ['S', 'Br', 'H', 'P', 'O', 'B', 'Si', 'N', 'C', 'I', 'F', 'Cl']
# The chemical formula is represented according to Hill convention, that is, 
# - beginning with carbon atoms
# - then hydrogens
# - then all other elements in alphabetical order
def identify_relevant_elements(df):
    tmp_chem_list = df["inchi_chemical_formula"].apply(lambda x: re.split(r'(\d+)', x))
    tmp_chem_list = [x for x in flatten_l_o_l(tmp_chem_list) if (not x.isnumeric() and x!="")]
    return list(set(flatten_l_o_l([re.findall('[A-Z][^A-Z]*',x) for x in tmp_chem_list])))


def chem_form_list_to_columns(chem_form_list, all_possible_elem_list):
    all_possible_elems = {e:0 for e in all_possible_elem_list}
    for e, val in chem_form_list:
        if val=="":
            val=1
        else:
            val=int(val)
        all_possible_elems[e]=val
    return [all_possible_elems[e] for e in all_possible_elem_list]


def tf_load_image(path, img_size=(224,224), tile_to_3_channel=True, for_tpu=False):
    img = decode_img(tf.io.read_file(path), img_size, n_channels=1, for_tpu=for_tpu)
    if tile_to_3_channel:
        return tf.tile(img, tf.constant((1, 1, 3), dtype=tf.int32))
    else:
        return img

def decode_img(img, img_size=(224,224), n_channels=1, for_tpu=False):
    """TBD"""
    
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=n_channels)

    # resize the image to the desired size
    if for_tpu:
        return tf.cast(tf.image.resize(img, img_size), tf.float32)
    else:
        return tf.cast(tf.image.resize(img, img_size), tf.uint8)
    
    
def convert_pred_batch_to_str_batch(pred_batch):
    return ["".join([
        f"{e}{int(c)}" if c>1 else f"{e}" \
        for (c, e) in [
            *zip(pred[np.nonzero(pred)], np.array(TRAIN_ELEMENTS)[np.nonzero(pred)])
        ]]) for pred in np.round(pred_batch)]

In [ ]:
train_df["prefixes"] = train_df.InChI.apply(lambda x: [xx[0] for xx in x.split("/")[2:]])
prefix_combos = train_df["prefixes"].apply(lambda x: "".join(x)).value_counts()
display(prefix_combos)

# Housekeeping Stuff
train_df["inchi_split"] = train_df.InChI.str.split("/")
train_df["inchi_root"] = train_df.inchi_split.apply(lambda x: x.pop(0))

# Guaranteed Stuff
train_df["inchi_chemical_formula"] = train_df.inchi_split.apply(lambda x: x.pop(0))
train_df["inchi_atom_connections_/c"] = train_df.inchi_split.apply(lambda x: get_sublayer(x, "c"))

# Optional Stuff - Note there are no included /q and /p layers
train_df["inchi_hydrogen_atoms_/h"] = train_df.inchi_split.apply(lambda x: get_sublayer(x, "h"))
train_df["inchi_charge_/q"] = None
train_df["inchi_proton_/p"] = None

train_df["inchi_db_and_c_/b"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "b"))
train_df["inchi_tetrahedral_/t"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "t"))
train_df["inchi_tetrahedral_/m"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "m"))
train_df["inchi_stereo_info_/s"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "s"))

train_df["inchi_isotopic_/s"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "i"))
train_df["inchi_isotopic_/h"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "h"))

train_df["inchi_tetrahedral_secondary_/t"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "t"))
train_df["inchi_tetrahedral_secondary_/m"] = train_df.inchi_split.apply(lambda x: get_sublayer_hard(x, "m"))

train_df = train_df.drop(columns=["inchi_split"])

# Non NAN Counts
display(train_df.count())
display(train_df.head())

train_df["chemical_formula_list"] = train_df["inchi_chemical_formula"].apply(lambda x: re.findall(r'([A-Z][a-z]*)([0-9]*)', x))
train_df["chemical_formula_list"] = train_df["chemical_formula_list"].apply(lambda x: chem_form_list_to_columns(x, TRAIN_ELEMENTS))
train_df = pd.concat([train_df, pd.DataFrame(train_df["chemical_formula_list"].to_list(), columns=[f"{e}_count" for e in TRAIN_ELEMENTS])], axis=1)
train_df = train_df.drop(columns=["chemical_formula_list"])

display(train_df)
display(train_df[["inchi_chemical_formula"]+[f"{e}_count" for e in TRAIN_ELEMENTS]])

<h1 style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="examine_identicals">4&nbsp;&nbsp;INVESTIGATION OF IDENTICAL MOLECULAR STRUCTURES</h1>

In [ ]:
repeated_formulas = [k for k,v in train_df["inchi_chemical_formula"].value_counts().items() if v>1]
demo_idx = 0
demo_formula = repeated_formulas[demo_idx]
train_df_formula_subset = train_df[train_df["inchi_chemical_formula"]==demo_formula].reset_index(drop=True)
row_to_display = 4

plt.figure(figsize=(18,5*row_to_display))
plt.suptitle(f"{demo_formula}", fontsize=20)
for i in range(min(4*row_to_display, len(train_df_formula_subset))):
    plt.subplot(5,4,i+1)
    plt.imshow(np.asarray(Image.open(train_df_formula_subset.img_path[i])), cmap="gray")
    plt.title("{}".format('\n'.join(train_df_formula_subset.InChI[i].split('/')[2:])), fontweight="bold", fontsize=9)
    
plt.tight_layout()
plt.show()

In [ ]:
repeated_formulas = [k for k,v in train_df["inchi_chemical_formula"].value_counts().items() if v>1]
demo_idx = 10
demo_formula = repeated_formulas[demo_idx]
train_df_formula_subset = train_df[train_df["inchi_chemical_formula"]==demo_formula].reset_index(drop=True)
row_to_display = 4

plt.figure(figsize=(18,5*row_to_display))
plt.suptitle(f"{demo_formula}", fontsize=20)
for i in range(min(4*row_to_display, len(train_df_formula_subset))):
    plt.subplot(5,4,i+1)
    plt.imshow(np.asarray(Image.open(train_df_formula_subset.img_path[i])), cmap="gray")
    plt.title("{}".format('\n'.join(train_df_formula_subset.InChI[i].split('/')[2:])), fontweight="bold", fontsize=9)
    
plt.tight_layout()
plt.show()

<h1 style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="baseline_model">5&nbsp;&nbsp;BASELINE MODEL</h1>

In [ ]:
train_df.image_id=train_df.image_id.astype(str)
train_df.img_path=train_df.img_path.astype(str)
train_df.inchi_chemical_formula=train_df.inchi_chemical_formula.astype(str)
_ = [train_df[c].fillna(0, inplace=True) for c in train_df.columns if c.endswith("_count")]

VAL_SPLIT = 0.05
VAL_INDICES = random.sample(range(len(train_df)), round(len(train_df)*VAL_SPLIT))

val_df = train_df.iloc[VAL_INDICES]
train_df = train_df[~train_df.index.isin(VAL_INDICES)]

feature_columns = ["image_id", "img_path"]
label_columns = [f"{e}_count" for e in TRAIN_ELEMENTS]

train_feature_ds = tf.data.Dataset.zip(tuple([tf.data.Dataset.from_tensor_slices(train_df[x].values) for x in feature_columns]))
train_label_ds = tf.data.Dataset.from_tensor_slices(train_df[label_columns].values)
train_ds = tf.data.Dataset.zip((train_feature_ds,train_label_ds))

val_feature_ds = tf.data.Dataset.zip(tuple([tf.data.Dataset.from_tensor_slices(val_df[x].values) for x in feature_columns]))
val_label_ds = tf.data.Dataset.from_tensor_slices(val_df[label_columns].values)
val_ds = tf.data.Dataset.zip((val_feature_ds,val_label_ds))

del train_label_ds, train_feature_ds; gc.collect(); gc.collect();
del val_label_ds, val_feature_ds; gc.collect(); gc.collect();

print(train_ds)
print(val_ds)

In [ ]:
MODEL_CKPT_DIR = "/kaggle/working/eb0_ckpts"

# BATCH_SIZE=BATCH_SIZE_PER_REPLICA*N_REPLICAS
BATCH_SIZE = 64
AUTOTUNE   = tf.data.AUTOTUNE
SHUFF_BUFF = 256

# Drop Image ID for now and open the image and stack y values as a single vector
# if N_REPLICAS==8: # TPU PATH
#     SAVE_LOCAL = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
#     train_ds = train_ds.map(lambda x,y: ((tf_load_image(x[1], for_tpu=True)), tf.cast(y, tf.float32)))
#     val_ds = val_ds.map(lambda x,y: ((tf_load_image(x[1], for_tpu=True)), tf.cast(y, tf.float32)))
# else:
# TRAIN_CACHE_DIR = "/kaggle/train_cache"
# VAL_CACHE_DIR = "/kaggle/val_cache"
# if not os.path.isdir(TRAIN_CACHE_DIR):
#     os.makedirs(TRAIN_CACHE_DIR, exist_ok=True)
# if not os.path.isdir(VAL_CACHE_DIR):
#     os.makedirs(VAL_CACHE_DIR, exist_ok=True)
SAVE_LOCAL=None
train_ds = train_ds.map(lambda x,y: ((tf_load_image(x[1])), tf.cast(y, tf.int32)))#.cache(TRAIN_CACHE_DIR)
val_ds = val_ds.map(lambda x,y: ((tf_load_image(x[1])), tf.cast(y, tf.int32)))#.cache(VAL_CACHE_DIR)

train_ds = train_ds.shuffle(SHUFF_BUFF) \
                   .batch(BATCH_SIZE) \
                   .prefetch(AUTOTUNE)

val_ds = val_ds.batch(BATCH_SIZE) \
               .prefetch(AUTOTUNE)

print(train_ds, val_ds)

In [ ]:
N_EPOCHS=4
LR_START = 0.0005
LR_MAX = 0.0015
LR_MIN = 0.00075
LR_RAMPUP_EPOCHS = 2
LR_SUSTAIN_EPOCHS = 1
LR_EXP_DECAY = 0.4

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

# VIEW SCHEDULE
rng = [i for i in range(N_EPOCHS)]
y = [lrfn(x) for x in rng]

plt.figure(figsize=(10,4))
plt.plot(rng, y)
plt.title("CUSTOM LR SCHEDULE", fontweight="bold")
plt.show()

print(f"Learning rate schedule: {y[0]:.3g} to {max(y):.3g} to {y[-1]:.3g}")

In [ ]:
def get_backbone(efficientnet_name="efficientnet_b0", input_shape=(224,224,3), include_top=False, weights="imagenet", pooling="avg"):
    if "b0" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB0(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b1" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB1(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b2" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB2(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b3" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB3(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b4" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB4(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b5" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB5(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b6" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB6(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b7" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB7(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    else:
        raise ValueError("Invalid EfficientNet Name!!!")
    return eb


def add_head_to_bb(bb, n_outputs=19, dropout=0.05, head_layer_nodes=(512,)):
    x = tf.keras.layers.BatchNormalization()(bb.output)
    x = tf.keras.layers.Dropout(dropout)(x)
    
    for n_nodes in head_layer_nodes:
        x = tf.keras.layers.Dense(n_nodes, activation="relu")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout/2)(x)
    
    output = tf.keras.layers.Dense(n_outputs, activation="linear")(x)
    return tf.keras.Model(inputs=bb.inputs, outputs=output)

# tf.keras.backend.clear_session()
# eb0 = get_backbone("b0")
# eb0 = add_head_to_bb(eb0, len(label_columns), dropout=0.4)
# eb0.compile(optimizer="adam", loss=tf.keras.losses.Huber(), metrics=["acc", "mae"])

# WE SKIP LOADING FRESH FOR DEMO PURPOSES
# tf.keras.backend.clear_session()
eb0 = tf.keras.models.load_model("../input/moleculartranslationbaselinemodel/eb0_ckpts/ckpt-0001-2.0007.ckpt")
tf.keras.utils.plot_model(eb0, show_shapes=True, show_dtype=True, dpi=55)

In [ ]:
# history = eb0.fit(
#     train_ds.take(20), # DEMO PURPOSES
#     validation_data=val_ds.take(10), # LEAVE AS FULL VAL SET FOR DEMO TO SEE PERFORMANCE ON VAL *~7-10 minutes~*
#     callbacks=[
#         tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
#         tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(MODEL_CKPT_DIR, "ckpt-{epoch:04d}-{val_loss:.4f}.ckpt"), verbose=1),
#         tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
#     ], 
#     epochs=1, # DEMO PURPOSES
# )
# eb0.save(os.path.join(MODEL_CKPT_DIR, "final"))
# eb0.evaluate(val_ds.take(10))

In [ ]:
repeated_formulas = [k for k,v in train_df["inchi_chemical_formula"].value_counts().items() if v>1]
demo_idx = 2
demo_formula = repeated_formulas[demo_idx]
train_df_formula_subset = train_df[train_df["inchi_chemical_formula"]==demo_formula].reset_index(drop=True)

batch_to_infer_on = tf.stack([tf_load_image(path) for path in train_df_formula_subset.img_path.values[:16]])
print(f"\n... GT FORMULA = {demo_formula} ...\n")
preds = eb0(batch_to_infer_on)

print(f"\nRANDOM STRING HAS A LEVENSHTEIN DISTANCE: {distance(demo_formula, 'C1H1O1N10Br4')}\n" \
      f"\t--> DEMO={demo_formula}\n" \
      f"\t--> RANDOM=C1HNO10Br4\n\n")
for i, y_pred in enumerate(convert_pred_batch_to_str_batch(preds)):
    print(f"PRED {i+1} HAS A LEVENSHTEIN DISTANCE OF {distance(y_pred, demo_formula)}\n" \
          f"\t-->DEMO={demo_formula}\n\t-->PRED={y_pred}\n")

In [ ]:
# Capture the image ids (paths) for the test set
ss_image_paths =  ss_df.image_id.apply(lambda x: os.path.join(TEST_DIR, x[0], x[1], x[2], x+".png")).values

test_feature_ds = tf.data.Dataset.from_tensor_slices(ss_image_paths)
test_ds = test_feature_ds.map(tf_load_image)
test_ds = test_ds.batch(BATCH_SIZE)

In [ ]:
# Iterate over the submission dataframe and generate predictions for chemical formula
chem_formula_preds = np.zeros((len(ss_image_paths,))).astype(str)
for i, tf_batch in enumerate(tqdm(test_ds)):
    chem_formula_preds[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = convert_pred_batch_to_str_batch(eb0(tf_batch))
    if i%1000==0:
        tf.keras.backend.clear_session()
        eb0 = tf.keras.models.load_model("../input/moleculartranslationbaselinemodel/eb0_ckpts/ckpt-0001-2.0007.ckpt")
        gc.collect(); gc.collect();
        
with open('/kaggle/working/preds.pickle', 'wb') as handle:
    pickle.dump(chem_formula_preds, handle)


# with open('../input/molecular-translation-eda-smart-baseline/preds.pickle', 'rb') as f:
#     chem_formula_preds = pickle.load(f)

In [ ]:
len(chem_formula_preds)

In [ ]:
chem_formula_preds = [str(x) for x in chem_formula_preds]
ss_df["InChI_build"] = [f"InChI=1S/{x}/c"+x+"/c" for x in chem_formula_preds]

In [ ]:
ss_df

In [ ]:
common_C = train_df["inchi_atom_connections_/c"].value_counts(dropna=False).index[0]
common_H = train_df["inchi_hydrogen_atoms_/h"].value_counts(dropna=False).index[0]
common_B = train_df["inchi_db_and_c_/b"].value_counts(dropna=False).index[0]
common_T1 = train_df["inchi_tetrahedral_/t"].value_counts(dropna=False).index[0]
common_M1 = train_df["inchi_tetrahedral_/m"].value_counts(dropna=False).index[0]
common_S1 = train_df["inchi_stereo_info_/s"].value_counts(dropna=False).index[0]
common_S2 = train_df["inchi_isotopic_/s"].value_counts(dropna=False).index[0]
common_H = train_df["inchi_isotopic_/h"].value_counts(dropna=False).index[0]
common_T2 = train_df["inchi_tetrahedral_secondary_/t"].value_counts(dropna=False).index[0]
common_M2 = train_df["inchi_tetrahedral_secondary_/m"].value_counts(dropna=False).index[0]

for x in [common_C, common_H, common_B, common_T1, common_M1, common_S1, common_S2, common_H, common_T2, common_M2,]: print(x)

In [ ]:
ss_df["InChI"] = ss_df["InChI_build"]+common_C
ss_df = ss_df.drop(columns=["InChI_build"])
ss_df

In [ ]:
# inchi_submissions = []
# for pred in chem_formula_preds:
#     inchi_submissions.append(train_df.iloc[np.argmin((train_df[[f"{e}_count" for e in TRAIN_ELEMENTS]]-pred).abs().sum(axis=1))].InChI)

In [ ]:
ss_df.to_csv("submission.csv", index=False)